In [ ]:
%pip install seaborn
%pip install folium
import pandas as pd # data wrangler library, dataframes are used to display and manipulate data
import seaborn as sns # data graphing library, built on top of matplotlib
import matplotlib.pyplot as plt # graphing library, used for titles and customization
import urllib.parse # library to parse URLs for querying
import seaborn as sns # data graphing library, built on top of matplotlib
import folium # library to create interactive maps
import folium.plugins as plugins # plugins for folium, used for clustering map points
import geopandas as gpd # library to handle geospatial data
from shapely.geometry import LineString
from pathlib import Path

In [4]:
def encode_soql_query(query:str) -> str:
    # just in case there are any new lines in the query, replaces them with spaces
    while '\n' in query:
        query = query.replace('\n', ' ')

    # encodes the query to be URL friendly
    parsed_query = urllib.parse.quote(query)

    # return query with prefix needed for soql queries
    return '?$query=' + parsed_query

In [10]:
ace_violations_api = "https://data.ny.gov/resource/kh8p-hcbm.csv"
class keyword_map:
  violation_final = ""
  def __init__(self, violation_search_keyword):
    self.violation_final = violation_search_keyword
    violations_keyword_query =  """
    SELECT  *
    WHERE violation_status = 'EXEMPT - BUS/PARATRANSIT'
    LIMIT 250
    """

  def encoding(keyword):
    encoded_query = encode_soql_query(keyword)
    api_query = ace_violations_api + encoded_query

    bus_keyword_df = pd.read_csv(api_query)
    geometry = gpd.points_from_xy(bus_keyword_df.bus_stop_longitude, bus_keyword_df.bus_stop_latitude)
    geo_df = gpd.GeoDataFrame(
    bus_keyword_df, geometry=geometry
    )

  def generate_no_subway_and_save(violations_keyword_query):
    encoding(violations_keyword_query)

    map = folium.Map(location=[40.730610, -73.935242], tiles="CartoDB Positron", zoom_start=11)

    unique_stops = geo_df.bus_route_id.nunique()
    color_palette = sns.color_palette("hls", unique_stops).as_hex()
    unique_values = geo_df.bus_route_id.unique()
    color_map = dict(zip(unique_values, color_palette))
    geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

    i = 0
    for coordinates in geo_df_list:
       map.add_child(
          folium.Marker(
            location=coordinates,
            popup=f"""
            Route: {geo_df.bus_route_id[i]} <br>
            Stop Name: {geo_df.stop_name[i]} <br>
            Violation Type: {geo_df.violation_type[i]} <br>
            """,
            icon=folium.Icon(color='white', icon_color = color_map[geo_df.bus_route_id[i]], icon="info-sign"),
            )
       )
       i = i + 1
    map.save('../data/cleaned/'+violation_final+'_map'+'.html')

  def subway_addition(map):
    FOLDER = Path("../data/gtfs_subway")
    print(f"Loading GTFS data from: {FOLDER.resolve()}")

    shapes_path = FOLDER / "shapes.txt"
    stops_path = FOLDER / "stops.txt"
    routes_path = FOLDER / "routes.txt"
    trips_path = FOLDER / "trips.txt"

    shapes = pd.read_csv(shapes_path, dtype=str, low_memory=False)
    stops = pd.read_csv(stops_path, dtype=str, low_memory=False)
    routes = pd.read_csv(routes_path, dtype=str, low_memory=False)
    trips = pd.read_csv(trips_path, dtype=str, low_memory=False)
    print("Successfully loaded GTFS text files.")

    feed_name = "subway"
    for df in [shapes, stops, routes, trips]:
      df["feed_name"] = feed_name

    for col in ["shape_pt_lat", "shape_pt_lon"]:
      shapes[col] = shapes[col].astype(float)
    shapes["shape_pt_sequence"] = shapes["shape_pt_sequence"].astype(int)
    stops["stop_lat"] = stops["stop_lat"].astype(float)
    stops["stop_lon"] = stops["stop_lon"].astype(float)

    shapes["shape_uid"] = shapes["feed_name"] + "_" + shapes["shape_id"]
    trips["shape_uid"] = trips["feed_name"] + "_" + trips["shape_id"]

    shape_to_route = trips.merge(
      routes, on=["route_id", "feed_name"]
    ).drop_duplicates("shape_uid")

    lines = (
      shapes.sort_values(["shape_uid", "shape_pt_sequence"])
      .groupby("shape_uid")[["shape_pt_lon", "shape_pt_lat"]]
      .apply(lambda df: LineString(df.to_numpy()))
      .to_frame("geometry")
      .reset_index()
    )

    routes_gdf = gpd.GeoDataFrame(lines, geometry="geometry", crs="EPSG:4326").merge(
      shape_to_route, on="shape_uid", how="left"
    )

    stops_gdf = gpd.GeoDataFrame(
      stops,
      geometry=gpd.points_from_xy(stops["stop_lon"], stops["stop_lat"]),
      crs="EPSG:4326"
    )

    folium.map.CustomPane("routes", z_index=400).add_to(map)
    folium.map.CustomPane("stops", z_index=650).add_to(map)

    folium.GeoJson(
      routes_gdf,
      name="Subway Routes",
      style_function=lambda feature: {
        'color': f"#{feature['properties'].get('route_color', '000000')}",
        'weight': 3,
        'opacity': 0.7,
      },
      tooltip=folium.GeoJsonTooltip(fields=["route_short_name", "route_long_name"]),
      pane="routes"
    ).add_to(map)

    folium.LayerControl(collapsed=False).add_to(map)




